In [ ]:
import laspy as las
import numpy as np
import random
from skspatial.objects import Line

ASSETS_FOLDER = 'assets/'
OUTPUT_FOLDER = 'output/'
FILENAME = 'lasfile.las'
OUTPUT_FILENAME = 'output'
THR = 600
K = 80

In [ ]:
def get_points(file):
  print('GETTING POINTS')
  points = np.vstack((file.X, file.Y, file.Z, file.classification)).transpose()

  minimum = min(file.Z)

  filtered_points = [point for point in points if point[3] < 2]
  remaining_points = [point for point in points if point[3] >= 2]

  remaining_points.extend([point for point in filtered_points if point[2] <= (minimum * 1.05)])
  filtered_points = [point for point in filtered_points if point[2] > (minimum * 1.05)]

  N = int(len(filtered_points) * random.uniform(0.005, 0.02))
  random.shuffle(filtered_points)
  
  N_points = filtered_points[0:N]
  other_points = filtered_points[N:]

  print(len(points), len(N_points) + len(remaining_points) + len(other_points))

  return filtered_points, N_points, other_points, remaining_points

In [ ]:
def fit(N_points, other_points, filtered_points, errors, N_points_array, other_points_array):
  print('FITTING')
  line_fit = Line.best_fit(list(map(lambda point: [point[0], point[1], point[2]], N_points)))
  new_other_points = []

  for point in list(other_points):
    distance = line_fit.distance_point([point[0], point[1], point[2]])

    if(distance < THR):
      N_points.append(point)
    else:
      new_other_points.append(point)

  print(len(N_points) / len(filtered_points))

  if len(N_points) / len(filtered_points) > 0.1:
    print('FOUND FIT')
    line_fit = Line.best_fit(list(map(lambda point: [point[0], point[1], point[2]], N_points)))
    err = 0.0

    for point in N_points:
      err += line_fit.distance_point([point[0], point[1], point[2]])

    print('ERRORS CALCULATED')
    errors.append(err)
    N_points_array.append(N_points)
    other_points_array.append(new_other_points)

In [ ]:
file = las.read(ASSETS_FOLDER + FILENAME)
errors = []
N_points_array = []
other_points_array = []

for k in range(K):
  print(F"K = {k + 1}")
  filtered_points, N_points, other_points, remaining_points = get_points(file)

  fit(N_points, other_points, filtered_points, errors, N_points_array, other_points_array)

  if len(errors) > 0:
    min_index = np.argmin(errors)
    filename = OUTPUT_FOLDER + OUTPUT_FILENAME + " " + str(errors[min_index]) + ".las"

    print(f'SAVING TO FILE: {filename}')

    all_points = list(map(lambda point: [point[0], point[1], point[2], 16], N_points_array[min_index])).extend(other_points_array[min_index]).extend(remaining_points)

    output_file = las.LasData(file.header)
    output_file.points = file.points.copy()
    output_file.X = list(map(lambda point: point[0], all_points))
    output_file.Y = list(map(lambda point: point[1], all_points))
    output_file.Z = list(map(lambda point: point[2], all_points))
    output_file.classification = list(map(lambda point: point[3], all_points))

    output_file.write(filename)